# DAT405/DIT407 Introduction to Data Science and AI 
## 2022-2023, Reading Period 4
## Assignment 5: Reinforcement learning and classification

Hints:
You can execute certain linux shell commands by prefixing the command with `!`. You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results the second you can use writing code snippets that execute the tasks required.  

This assignment is about **sequential decision making** under uncertainty (Reinforcement learning). In a sequential decision process, the process jumps between different states (the environment), and in each state the decision maker, or agent, chooses among a set of actions. Given the state and the chosen action, the process jumps to a new state. At each jump the decision maker receives a reward, and the objective is to find a sequence of decisions (or an optimal policy) that maximizes the accumulated rewards.

We will use **Markov decision processes** (MDPs) to model the environment, and below is a primer on the relevant background theory. 



* To make things concrete, we will first focus on decision making under **no** uncertainity (question 1 and 2), i.e, given we have a world model, we can calculate the exact and optimal actions to take in it. We will first introduce **Markov Decision Process (MDP)** as the world model. Then we give one algorithm (out of many) to solve it.

* (Optional) Next we will work through one type of reinforcement learning algorithm called Q-learning (question 3). Q-learning is an algorithm for making decisions under uncertainity, where uncertainity is over the possible world model (here MDP). It will find the optimal policy for the **unknown** MDP, assuming we do infinite exploration.

* Finally, in question 4 you will be asked to explain differences between reinforcement learning and supervised learning and in question 5 write about decision trees and random forests.

## Primer
### Decision Making
The problem of **decision making under uncertainty** (commonly known as **reinforcement learning**) can be broken down into
two parts. First, how do we learn about the world? This involves both the
problem of modeling our initial uncertainty about the world, and that of drawing conclusions from evidence and our initial belief. Secondly, given what we
currently know about the world, how should we decide what to do, taking into
account future events and observations that may change our conclusions?
Typically, this will involve creating long-term plans covering possible future
eventualities. That is, when planning under uncertainty, we also need to take
into account what possible future knowledge could be generated when implementing our plans. Intuitively, executing plans which involve trying out new
things should give more information, but it is hard to tell whether this information will be beneficial. The choice between doing something which is already
known to produce good results and experiment with something new is known
as the **exploration-exploitation dilemma**.

### The exploration-exploitation trade-off

Consider the problem of selecting a restaurant to go to during a vacation. Lets say the
best restaurant you have found so far was **Les Epinards**. The food there is
usually to your taste and satisfactory. However, a well-known recommendations
website suggests that **King’s Arm** is really good! It is tempting to try it out. But
there is a risk involved. It may turn out to be much worse than **Les Epinards**,
in which case you will regret going there. On the other hand, it could also be
much better. What should you do?
It all depends on how much information you have about either restaurant,
and how many more days you’ll stay in town. If this is your last day, then it’s
probably a better idea to go to **Les Epinards**, unless you are expecting **King’s
Arm** to be significantly better. However, if you are going to stay there longer,
trying out **King’s Arm** is a good bet. If you are lucky, you will be getting much
better food for the remaining time, while otherwise you will have missed only
one good meal out of many, making the potential risk quite small.

### Markov Decision Processes
Markov Decision Processes (MDPs) provide a mathematical framework for modeling sequential decision making under uncertainty. An *agent* moves between *states* in a *state space* choosing *actions* that affects the transition probabilities between states, and the subsequent *rewards* recieved after a jump. This is then repeated a finite or infinite number of epochs. The objective, or the *solution* of the MDP, is to optimize the accumulated rewards of the process.

Thus, an MDP consists of five parts: 

* Decision epochs: $t={1,2,...,T}$, where $T\leq \infty$
* State space: $S=\{s_1,s_2,...,s_N\}$ of the underlying environment
* Action space $A=\{a_1,a_2,...,a_K\}$ available to the decision maker at each decision epoch
* Transition probabilities $p(s_{t+1}|s_t,a_t)$ for jumping from state $s_t$ to state $s_{t+1}$ after taking action $a_t$
* Reward functions $R_t = r(a_t,s_t,s_{t+1})$ resulting from the chosen action and subsequent transition

A *decision policy* is a function $\pi: s \rightarrow a$, that gives instructions on what action to choose in each state. A policy can either be *deterministic*, meaning that the action is given for each state, or *randomized* meaning that there is a probability distribution over the set of possible actions for each state. Given a specific policy $\pi$ we can then compute the the *expected total reward* when starting in a given state $s_1 \in S$, which is also known as the *value* for that state, 

$$V^\pi (s_1) = E\left[ \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) {\Large |} s_1\right] = \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) p(s_{t+1} | a_t,s_t)$$ 

where $a_t = \pi(s_t)$. To ensure convergence and to control how much credit to give to future rewards, it is common to introduce a *discount factor* $\gamma \in [0,1]$. For instance, if we think all future rewards should count equally, we would use $\gamma = 1$, while if we value near-future rewards higher than more distant rewards, we would use $\gamma < 1$. The expected total *discounted* reward then becomes

$$V^\pi( s_1) = \sum_{t=1}^T \gamma^{t-1} r(s_t,a_t, s_{t+1}) p(s_{t+1} | s_t, a_t) $$

Now, to find the *optimal* policy we want to find the policy $\pi^*$ that gives the highest total reward $V^*(s)$ for all $s\in S$. That is, we want to find the policy where

$$V^*(s) \geq V^\pi(s), s\in S$$

To solve this we use a dynamic programming equation called the *Bellman equation*, given by

$$V(s) = \max_{a\in A} \left\{\sum_{s'\in S} p(s'|s,a)( r(s,a,s') +\gamma V(s')) \right\}$$

It can be shown that if $\pi$ is a policy such that $V^\pi$ fulfills the Bellman equation, then $\pi$ is an optimal policy.

A real world example would be an inventory control system. The states could be the amount of items we have in stock, and the actions would be the amount of items to order at the end of each month. The discrete time would be each month and the reward would be the profit. 


## Question 1

The first question covers a deterministic MPD, where the action is directly given by the state, described as follows:

* The agent starts in state **S** (see table below)
* The actions possible are **N** (north), **S** (south), **E** (east), and **W** west. 
* The transition probabilities in each box are deterministic (for example P(s'|s,N)=1 if s' north of s). Note, however, that you cannot move outside the grid, thus all actions are not available in every box.
* When reaching **F**, the game ends (absorbing state).
* The numbers in the boxes represent the rewards you receive when moving into that box. 
* Assume no discount in this model: $\gamma = 1$
    
    
| | | |
|----------|----------|---------|
|-1 |1|**F**|
|0|-1|1|  
|-1 |0|-1|  
|**S**|-1|1|

Let $(x,y)$ denote the position in the grid, such that $S=(0,0)$ and $F=(2,3)$.


**1a)** What is the optimal path of the MDP above? Is it unique? Submit the path as a single string of directions. E.g. NESW will make a circle.

Answer 1a:

The optimal path from "S" to "F" is "EENNN" which results in a total reward of 0. There are alternative paths that have the same total reward, such as "EENNWENE", hence the solution is not unique. However this path is longer.

**1b)** What is the optimal policy (i.e. the optimal action in each state)? It is helpful if you draw the arrows/letters in the grid.

Answer 1b:

When selecting a policy, we aim to find a $\pi$ that maximizes the expected cumulative reward, as represented by the value function $V$. The optimal policy for each state is given in the Table below:

State |Optimal Policy ($\pi^*$)|Reward ($V$)
------|----------------------|------
(0,0) |E or N                |-1
(1,0) |N or E                |0
(2,0) |N, E or S             |-1
(3,0) |E                     |1
(0,1) |E                     |1
(0,2) |N or W                |-1
(1,1) |N, E, W or S          |-1
(1,2) |N or S                |1
(2,1) |E or N                |1
(2,2) |N                     |Finish
(3,1) |E                     |Finish
(3,2) |Finish state          |-

**1c)** What is expected total reward for the policy in 1a)?

Answer 1c:

The expected total reward is 0.

## Value Iteration

For larger problems we need to utilize algorithms to determine the optimal policy $\pi^*$. *Value iteration* is one such algorithm that iteratively computes the value for each state. Recall that for a policy to be optimal, it must satisfy the Bellman equation above, meaning that plugging in a given candidate $V^*$ in the right-hand side (RHS) of the Bellman equation should result in the same $V^*$ on the left-hand side (LHS). This property will form the basis of our algorithm. Essentially, it can be shown that repeated application of the RHS to any intial value function $V^0(s)$ will eventually lead to the value $V$ which statifies the Bellman equation. Hence repeated application of the Bellman equation will also lead to the optimal value function. We can then extract the optimal policy by simply noting what actions that satisfy the equation.    

The process of repeated application of the Bellman equation is what we here call the _value iteration_ algorithm. It practically procedes as follows:

```
epsilon is a small value, threshold
for x from i to infinity 
do
    for each state s
    do
        V_k[s] = max_a Σ_s' p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
    end
    if  |V_k[s]-V_k-1[s]| < epsilon for all s
        for each state s,
        do
            π(s)=argmax_a ∑_s′ p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
            return π, V_k 
        end
end

```

**Example:** We will illustrate the value iteration algorithm by going through two iterations. Below is a 3x3 grid with the rewards given in each state. Assume now that given a certain state $s$ and action $a$, there is a probability 0.8 that that action will be performed and a probability 0.2 that no action is taken. For instance, if we take action **E** in state $(x,y)$ we will go to $(x+1,y)$ 80 percent of the time (given that that action is available in that state), and remain still 20 percent of the time. We will use have a discount factor $\gamma = 0.9$. Let the initial value be $V^0(s)=0$ for all states $s\in S$. 

**Reward**:

| | | |  
|----------|----------|---------|  
|0|0|0|
|0|10|0|  
|0|0|0|  


**Iteration 1**: The first iteration is trivial, $V^1(s)$ becomes the $\max_a \sum_{s'} p(s'|s,a) r(s,a,s')$ since $V^0$ was zero for all $s'$. The updated values for each state become

| | | |  
|----------|----------|---------|  
|0|8|0|
|8|2|8|  
|0|8|0|  
  
**Iteration 2**:  
  
Staring with cell (0,0) (lower left corner): We find the expected value of each move:  
Action **S**: 0  
Action **E**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **N**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **W**: 0

Hence any action between **E** and **N** would be best at this stage.

Similarly for cell (1,0):

Action **N**: 0.8( 10 + 0.9 \* 2) + 0.2(0 + 0.9 \* 8) = 10.88 (Action **N** is the maximizing action)  

Similar calculations for remaining cells give us:

| | | |  
|----------|----------|---------|  
|5.76|10.88|5.76|
|10.88|8.12|10.88|  
|5.76|10.88|5.76|  


## Question 2

**2a)** Code the value iteration algorithm just described here, and show the converging optimal value function and the optimal policy for the above 3x3 grid. Make sure to consider that there may be several equally good actions for a state when presenting the optimal policy.

In [35]:
# Answer 2a
import numpy as np

# Parameters
gamma = 0.9
prob = 0.8
epsilon = 0.0001

# Initiate reward, state, next state and policy
reward = np.array([[0,0,0],
                 [0,10,0],
                 [0,0,0]], dtype=float)

state = np.array([[0,0,0],
                 [0,0,0],
                 [0,0,0]], dtype=float)

next_state = np.array([[0,0,0],
                 [0,0,0],
                 [0,0,0]], dtype=float)

policy = np.zeros(state.shape, dtype='object')

num_rows, num_col = state.shape

# Method that calculates the value using Bellman's equation
def calc_reward(current_reward, next_reward, current_state, next_state, gamma_value:float, probability:float):
    V = probability*(next_reward+gamma_value*next_state)+(1-probability)*(current_reward+gamma_value*current_state)
    return V


while True:
    # Loop over all elements in matrix
    for i in range(num_rows):

        for j in range(num_col):
            # Store temporary values and possible actions for each iteration
            temp_values = []
            action_list = []
            optimal_action = []

            # Action = N
            if i != 0:
                temp_values.append(calc_reward(reward[i,j], reward[i-1,j], state[i,j], state[i-1,j], gamma, prob))
                action_list.append('N')

            # Action = W
            if j != 0:
                temp_values.append(calc_reward(reward[i,j], reward[i,j-1], state[i,j], state[i,j-1], gamma, prob))
                action_list.append('W')

            # Action = E
            if j != (num_col - 1):
                temp_values.append(calc_reward(reward[i,j], reward[i,j+1], state[i,j], state[i,j+1], gamma, prob))
                action_list.append('E')

            # Action = S
            if i != (num_rows - 1):
                temp_values.append(calc_reward(reward[i,j], reward[i+1,j], state[i,j], state[i+1,j], gamma, prob))
                action_list.append('S')

            # Max value and best policy for each iteration are stored
            next_state[i,j] = max(temp_values)
            max_indices = [index for index, item in enumerate(temp_values) if item == max(temp_values)]
            for index in max_indices:
                optimal_action.append(action_list[index])
            optimal_action = ''.join(optimal_action)
            # Assign optimal policy (action) to policy matrix
            policy[i,j] = optimal_action


    diff = np.abs(state - next_state).sum()
    
    # Loop breaks when threshold is met
    if diff < epsilon:
        break
    
    # If threshold is not met update state matrix
    state = np.copy(next_state)
    

print(f'Converging optimal values: \n {state}\n')
print(f'Optimal policy: \n {policy}')




Converging optimal values: 
 [[45.61281773 51.94794601 45.61281773]
 [51.94794601 48.05184212 51.94794601]
 [45.61281773 51.94794601 45.61281773]]

Optimal policy: 
 [['ES' 'S' 'WS']
 ['E' 'NWES' 'W']
 ['NE' 'N' 'NW']]


**2b)** Explain why the result of 2a) does not depend on the initial value $V_0$.

Answer 2b: 

The rewards values decrease gradually at each iteration because of the gamma factor, $\gamma$. Independent of the initial value $V_0$, the optimal policy will ultimately converge to the same value. A larger $V_0$ value results in the algorithm requiring more iterations to converge.

**2c)** Describe your interpretation of the discount factor $\gamma$. What would happen in the two extreme cases $\gamma = 0$ and $\gamma = 1$? Given some MDP, what would be important things to consider when deciding on which value of $\gamma$ to use?

Answer 2c:

The discount factor $\gamma$ determines the relative importance of immediate and future rewards. It affects how much the agent prioritizes long-term rewards over short-term rewards.

A discount factor of 0 means that the agent only cares about immediate rewards and ignores all future rewards. A discount factor of 1 means that the agent values future rewards equally to immediate rewards.

The choice of $\gamma$ can also affect the speed of convergence and the stability of the algorithm. A higher $\gamma$ value can make the algorithm slower to converge but can also make it more stable, while a lower $\gamma$ value can make the algorithm faster to converge but less stable.

## Reinforcement Learning (RL) (Theory for optional question 3)
Until now, we understood that knowing the MDP, specifically $p(s'|a,s)$ and $r(s,a,s')$ allows us to efficiently find the optimal policy using the value iteration algorithm. Reinforcement learning (RL) or decision making under uncertainity, however, arises from the question of making optimal decisions without knowing the true world model (the MDP in this case).

So far we have defined the value function for a policy through $V^\pi$. Let's now define the *action-value function*

$$Q^\pi(s,a) = \sum_{s'} p(s'|a,s) [r(s,a,s') + \gamma V^\pi(s')]$$

The value function and the action-value function are directly related through

$$V^\pi (s) = \max_a Q^\pi (s,a)$$

i.e, the value of taking action $a$ in state $s$ and then following the policy $\pi$ onwards. Similarly to the value function, the optimal $Q$-value equation is:

$$Q^*(s,a) = \sum_{s'} p(s'|a,s) [r(s,a,s') + \gamma V^*(s')]$$

and the relationship between $Q^*(s,a)$ and $V^*(s)$ is simply

$$V^*(s) = \max_{a\in A} Q^*(s,a).$$

#### Q-learning

Q-learning is a RL-method where the agent learns about its unknown environment (i.e. the MDP is unknown) through exploration. In each time step *t* the agent chooses an action *a* based on the current state *s*, observes the reward *r* and the next state *s'*, and repeats the process in the new state. Q-learning is then a method that allows the agent to act optimally. Here we will focus on the simplest form of Q-learning algorithms, which can be applied when all states are known to the agent, and the state and action spaces are reasonably small. This simple algorithm uses a table of Q-values for each $(s,a)$ pair, which is then updated in each time step using the update rule in step $k+1$

$$Q_{k+1}(s,a) = Q_k(s,a) + \alpha \left( r(s,a) + \gamma \max \{Q_k(s',a')\} - Q_k(s,a) \right) $$ 

where $\gamma$ is the discount factor as before, and $\alpha$ is a pre-set learning rate. It can be shown that this algorithm converges to the optimal policy of the underlying MDP for certain values of $\alpha$ as long as there  is sufficient exploration. For our case, we set a constant $\alpha=0.1$.

#### OpenAI Gym

We shall use already available simulators for different environments (worlds) using the popular [OpenAI Gym library](https://www.gymlibrary.dev/). It just implements different types of simulators including ATARI games. Although here we will only focus on simple ones, such as the **Chain enviroment** illustrated below.
![alt text](https://chalmersuniversity.box.com/shared/static/6tthbzhpofq9gzlowhr3w8if0xvyxb2b.jpg)
The figure corresponds to an MDP with 5 states $S = \{1,2,3,4,5\}$ and two possible actions $A=\{a,b\}$ in each state. The arrows indicate the resulting transitions for each state-action pair, and the numbers correspond to the rewards for each transition.

## Question 3 (optional)
You are to first familiarize with the framework of [the OpenAI environments](https://www.gymlibrary.dev/), and then implement the Q-learning algorithm for the <code>NChain-v0</code> enviroment depicted above, using default parameters and a learning rate of $\gamma=0.95$. Report the final $Q^*$ table after convergence of the algorithm. For an example on how to do this, you can refer to the Q-learning of the **Frozen lake environment** (<code>q_learning_frozen_lake.ipynb</code>), uploaded on Canvas. Hint: start with a small learning rate.

Note that the NChain environment is not available among the standard environments, you need to load the <code>gym_toytext</code> package, in addition to the standard gym:

<code>
!pip install gym-legacy-toytext<br>
import gym<br>
import gym_toytext<br>
env = gym.make("NChain-v0")<br>
</code>

In [40]:
# Answer 3
import gym
import numpy as np
import random
import math
env = gym.make("NChain-v0")

num_episodes = 500 # 14000 Takes alot of time when increasing num_episodes
gamma = 0.95
learning_rate = 0.1
epsilon = 0.5

num_states = env.observation_space.n
num_actions = env.action_space.n

# Initialize the Q-table with the correct dimensions
Q = np.zeros((num_states, num_actions))

for episode in range(num_episodes):
    state = env.reset()
    done = False
    while done == False:
        # Select an action
        if random.uniform(0, 1) < epsilon:
            # Explore action space
            action = env.action_space.sample()
        else:
            # Exploit learned values
            action = np.argmax(Q[state, :])
        # Receive the feedback from the environment as a result of the action
        next_state, reward, done, info = env.step(action)
        # Learn from the experience by updating the Q-value of the action
        prediction_error = reward + (gamma*np.max(Q[next_state,:])) - Q[state, action]
        Q[state, action] += learning_rate*prediction_error
        state = next_state
        if done:
            break


print(Q)




[[54.39631341 54.45963418]
 [57.08604384 55.56405683]
 [59.19460704 55.93147353]
 [65.99879724 56.43940034]
 [73.85639911 61.04640407]]


## Question 4

**4a)** What is the importance of exploration in reinforcement learning? Explain with an example.

Answer 4a: Exploration is an important aspect of reinforcement learning. It allows you to get to know the environment by seeking new experiences and trying out different actions. Without doing this, you won't have any knowledge on what consequences your actions can bring. By exploring, you gain knowledge on which actions will improve your score the most. 

Especially important in cases where the environment is stochastic and unpredictable, exploration will help with getting to know the environment and avoid sticking with suboptimal policies. 

An example: Say you are in a maze attempting to find apples placed stochastically throughout the maze. At first, you may move around randomly attempting to gain knowledge of the maze and exploring different parts. You take note of which areas of the maze you find apples in while doing this. When exploring the maze, you slowly get to know the maze and start to know where you can find the most apples, where there are blind alleys etcetera. Simply put, by using exploration, you can think up a map of the maze in your head which then can be used to make informed decisions of where to move in order to maximize your reward (in this case apples). However, this can backfire in a way where you overexplore parts that you notice holds many apples, putting too little focus on unexplored parts which might contain even more apples and even better paths to take. 

**4b)** Explain what makes reinforcement learning different from supervised learning tasks such as regression or classification. 

Answer 4b: Reinforcement learning is more complex and dynamic compared to regression classification since it enables learning from experience and adapting to different environments to reach a goal. Reinforcement learning (RL) is goal orientated and doesn't simply give an output from an input like regression or classification, when using RL you are instead attempting to maximize a reward to reach your goal which makes it more dynamic. Instead of learning from a labeled dataset, RL interacts with the data and simulates the environment whicih then is used to make informed decisions towards the goal. Rl is not a supervised learning task, meaning you don't have to label the data. It learns from exploring and gaining rewards. 

## Question 5

**5a)** Give a summary of how a decision tree works and how it extends to random forests.

Answer 5a: A decision tree is a way to reach a decision on something based on yes or no questions. You start at a yes or no question,depending on your answer, you will end up up at a new question. The convention is to have questions as nodes and answers as arrows. You go down and left if the answer to the question is yes and down and right if the answer is no. Continue answering questions and moving further down until you reach the end. The question nodes will now have spanned a node tree. At the end of each path, you will reach a decision, based on the path you have taken in the tree. This is useful when you need to make a decision based on several linked questions.

This does work great with decision making on data that was used to create the decision tree. However, this is not very dynamic and therefore, attempting to classify new samples that is not from the data used to create the tree can result in inaccurate decisions. This is where random forests comes into play.

Random trees takes use of decision trees simplicity and improves it by accounting for randomness in the data, increasing the accuracy in the decision making. Random trees are created by taking your dataset and bootstrapping it by randomly picking lines from the original dataset. Next, randomly select questions that will be candidates for the root question, decide which question does the best job at separating the samples and select this as your root question. Continue considering different subsets of variables at each step until you have a tree. Then do this until you have a set of trees. These trees form a random forest. 

Now using our random forest, we run a set of data through all the trees in the forest. Count up which decision gets picked the most and you have your final decision! 

**5b)** State at least one advantage and one drawback with using random forests over decision trees.

Answer 5b: Random trees have the advantage of being more accurate and robust due to using multiple trees instead of only one. Using several trees canceles out biases that might be found in a single tree because of the redundancy it creates. A disadvantage is that random forests are slower since it requires generating a lot of trees and then also running the data through all of them, compared to only generating one and only running the data once. It is also harder to interpret because of this. A decision tree is easy to follow and decern what the final decision will be. In a random forest it's a more complex job .


# References
Primer/text based on the following references:
* http://www.cse.chalmers.se/~chrdimi/downloads/book.pdf
* https://github.com/olethrosdc/ml-society-science/blob/master/notes.pdf
* Question 2b: http://web4.cs.ucl.ac.uk/staff/D.Barber/textbook/090310.pdf
* Question 2c: https://stats.stackexchange.com/questions/221402/understanding-the-role-of-the-discount-factor-in-reinforcement-learning
* Question 3: "q_learning_intro - Jupyter Notebook.pdf" provided on canvas



